In [154]:
import torch
from torch.utils.data.dataset import Dataset, TensorDataset
from torch.utils.data.dataloader import DataLoader
from DiacriticDataset import DiacriticDatasetShaddah
import pickle
from collections import Counter
from itertools import product
from collections import defaultdict

In [155]:
train_dataset = DiacriticDatasetShaddah('Dataset_with_shaddah/train_cleaned_withshadda_215.txt','Dataset_with_shaddah/letter_to_id.pickle','Dataset_with_shaddah/id_to_letter.pickle','Dataset_with_shaddah/diacritic_to_id.pickle','Dataset_with_shaddah/id_to_diacritic.pickle','Dataset_with_shaddah/word_to_id.pickle','Dataset_with_shaddah/id_to_word.pickle','Dataset/diacritic_to_id.pickle','Dataset/id_to_diacritic.pickle','Dataset_with_shaddah/word_to_id_withdiacritics.pickle','Dataset_with_shaddah/id_to_word_withdiacritics.pickle')

In [156]:
train_dataloader = DataLoader(train_dataset)

In [157]:
trigram_counts = Counter()
bigram_counts = Counter()
unigram_counts = Counter()
counts = 0
letter_diacritic_counts = Counter()

In [158]:
for training_example in train_dataloader:
    letters = training_example[0][0].tolist()
    diacritics = training_example[1][0].tolist()
    for index,diacritic in enumerate(diacritics):
        if index==0:
            trigram_counts[('*','*',diacritics[index])] += 1
            bigram_counts[('*',diacritics[index])] += 1
            unigram_counts[diacritics[index]] += 1
        elif index==1:
            trigram_counts[('*',diacritics[index-1],diacritics[index])] += 1
            bigram_counts[(diacritics[index-1],diacritics[index])] += 1
            unigram_counts[diacritics[index]] += 1
        
        else:
            trigram_counts[(diacritics[index-2],diacritics[index-1],diacritics[index])] += 1
            bigram_counts[(diacritics[index-1],diacritics[index])] += 1
            unigram_counts[diacritics[index]] += 1
        

        letter_diacritic_counts[(diacritics[index],letters[index])] += 1
        
        counts += 1
    
    trigram_counts[(diacritics[-2],diacritics[-1],'STOP')] += 1
    bigram_counts[(diacritics[-1],'STOP')] += 1
    unigram_counts['STOP'] += 1
    counts += 1
    

In [159]:
transitions = Counter()
emissions = Counter()

In [160]:
l1 = (1/3)
l2 = (1/3)
l3 = (1/3)

In [161]:
diacrtic_ids = list(range(16))

In [162]:
for s in diacrtic_ids + ['STOP']:
    for (u,v) in product(diacrtic_ids+['*'],diacrtic_ids+['*']):
        if bigram_counts[(u,v)]>0:
            trigram_prob = trigram_counts[(u,v,s)]/bigram_counts[(u,v)]
        else:
            trigram_prob = 0
        
        
        if unigram_counts[v]>0:
            bigram_prob = bigram_counts[(v,s)]/unigram_counts[v]
        else:
            bigram_prob = 0
        
        unigram_prob = unigram_counts[s]/counts
        transitions[(s,u,v)] = l1 * trigram_prob + l2 * bigram_prob + l3 * unigram_prob

In [163]:
letters_ids = list(range(37))

In [164]:
for x in letters_ids:
    for s in diacrtic_ids:
        emissions[(x,s)] = (letter_diacritic_counts[(s,x)]/unigram_counts[s])

In [166]:
with open('HMM_Model/transition_probabilities','wb') as file:
    pickle.dump(transitions,file)

with open('HMM_Model/emission_probabilities','wb') as file:
    pickle.dump(emissions,file)